<a href="https://colab.research.google.com/github/shelleyg-bit/canada-land-cover-classifier/blob/main/gee_download_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install geopandas
!pip install geojson
!pip install geemap
!pip install geotable

In [3]:
import ee
ee.Authenticate() # give this notebook access to your google ee account
ee.Initialize() # this talks to ee backend to find out about all features of ee

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=4fAXSFWmvXX4R_iEpd6KqlYajp2eBa-ZjG5e63GsmOY&tc=CZ49neSoJmOpZh8X2wjj4Fr8DJmiFbALnKRP9vFMKXo&cc=_W71AN3HoH_M9_4LHxfhKG-VAMD4uh7KiUgyzgc1Ijk

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjGGkmZFEJHCjOwjVkaoXSzjHZ5JplF5N3CySUBzbvihf5hzgf4Vro

Successfully saved authorization token.


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/FarmHand/data_prep')

In [5]:
path1 ='/content/drive/MyDrive/FarmHand/data/All MB kml files/original_kmz_files/'
path2 ='/content/drive/MyDrive/farmhand/Data/farms_sheet.xlsx'
sheet ='Farm data (for farms created in'
path3 ='/content/drive/MyDrive/farmhand/Data/Mint farm data (2022 season)/Farmer data of mint. (1).xlsx'

In [6]:
from Kmz_n_excel_comb_2shp import comb_kmz_n_excel_2shp

df = comb_kmz_n_excel_2shp(path1, path2, sheet, path3)

shape file stored at the following location /content/drive/MyDrive/farmhand/Data/Mint farm data (2022 season) 


In [7]:
df.head()

,Name,geometry,class
0,K Shankar guru,"POLYGON Z ((76.73941 15.24347 0.00000, 76.7394...",Chilli-Non Farmhand
1,siddappa,"POLYGON Z ((76.71973 15.26482 0.00000, 76.7195...",Chilli-Non Farmhand
2,chandru nayaka,"POLYGON Z ((76.96205 15.12738 0.00000, 76.9616...",Chilli-Non Farmhand
3,Dodda Basava G,"POLYGON Z ((76.72902 15.24342 0.00000, 76.7290...",Chilli-Non Farmhand
4,udaya m,"POLYGON Z ((76.96406 15.12368 0.00000, 76.9636...",Chilli-Non Farmhand


In [8]:
df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
import geemap
import geopandas as gpd

import numpy as np

import matplotlib.pyplot as plt


## Create an EE Geometry Object for Area of Interest 



In [10]:
#Carefully choosen post analyzing
bbox1 =[76.65, 15.2,76.8, 15.37 ] #chilli farms aoi_1
bbox2 =[76.9, 15.05, 77.05, 15.15] #chilli farms aoi_2
bbox3 =[79.1996545, 28.4018788, 89.6, 28.75 ] #mint farms aoi_3

In [11]:
from shapely.geometry import box, Polygon, MultiPolygon

chilli_bbox1 = Polygon(box(*bbox1))
chilli_bbox2 = Polygon(box(*bbox2))
mint_bbox = Polygon(box(*bbox3))
aoi_gdf = gpd.GeoDataFrame({'geometry': [chilli_bbox1, chilli_bbox2, mint_bbox]}, crs=df.crs)
aoi = geemap.geopandas_to_ee(aoi_gdf)

In [12]:
aoi.getInfo()

{'columns': {'system:index': 'String'},
 'features': [{'geometry': {'coordinates': [[[76.65, 15.2],
      [76.8, 15.2],
      [76.8, 15.37],
      [76.65, 15.37],
      [76.65, 15.2]]],
    'type': 'Polygon'},
   'id': '0',
   'properties': {},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[[76.9, 15.05],
      [77.05, 15.05],
      [77.05, 15.15],
      [76.9, 15.15],
      [76.9, 15.05]]],
    'type': 'Polygon'},
   'id': '1',
   'properties': {},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[[79.1996545, 28.4018788],
      [89.6, 28.4018788],
      [89.6, 28.75],
      [79.1996545, 28.75],
      [79.1996545, 28.4018788]]],
    'type': 'Polygon'},
   'id': '2',
   'properties': {},
   'type': 'Feature'}],
 'type': 'FeatureCollection'}


## Get Image Collection from Sentinel

- For each harvest cycle(Sept-Feb) from 2019-2022, filtering out only harvest cycle months and discarding non-usable months (May-June)
- Collecting all the remaining raw bands on sentinel as @Mani Babu suggested. (right now I just collected 10m bands (RGB+NearIR) & NDVI)
- Combining All the bands for each harvest cycle in a single Tiff file.
- Monthly mean of every band
- A single tif will contain MonthName_Band_No as one band , so total number of bands will be  in one tiff will be Number of Months* Number of bands 
- Resulting in One GeoTiff file per harvest cycle
- Adding Mint farms to the collection set.

In [274]:
collection_sentinel = ('COPERNICUS/S2_SR')
time_range = ee.DateRange('2019-08-01', '2020-05-31')
start = time_range.start()
end = time_range.end()
count = end.difference(start, 'month').round().toInt()

In [275]:
count.getInfo()

10

In [276]:
month_seq = ee.List.sequence(0, count.subtract(1))

In [302]:
def obtain_image_sentinel_allbands(collection, time_range, area):
    """ Query satellite imagegry from GEE 
    - [B2, B3, B4, B8] - 10m resolution
    - use images only with <= 10% cloud-cover 
    from a collection of images in the Sentinel 2 dataset
    See also: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

    Args:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
         (ee.ImageCollection)
     """

    sentinel_filtered = (ee.ImageCollection(collection).
                       filterBounds(area))
                       #filterDate(time_range.start(), time_range.end()))
    

    return sentinel_filtered

In [303]:
images_collection = obtain_image_sentinel_allbands(collection_sentinel, time_range, area=aoi)
#images_collection = obtain_image_sentinel_allbands(collection_sentinel, time_range, area=aoi)

In [298]:
images_collection.size().getInfo()
#images_collection.getInfo()['Features'][1]

1858

In [337]:
endDate = start.advance(1, 'month')

def monthly_composite(start, end, reducer):
  # method to remove cloud from the image
  def maskclouds(image):
      """To mask clouds using the Sentinel-2 QA band
      @param {ee.Image} image Sentinel-2 image
      @return {ee.Image} cloud masked Sentinel-2 image
      """
      band_qa = image.select('QA60')
      cloud_mask = ee.Number(2).pow(10).int()
      cirrus_mask = ee.Number(2).pow(11).int()
      mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
          band_qa.bitwiseAnd(cirrus_mask).eq(0))
      return image.updateMask(mask).divide(10000)
  
  def fndvi(image):
      """Add NDVI to each pixel in an image"""
      #TODO: ask toby about this NDVI calculation
      # since its 20m resolution
      # there are other NDVIs at 10m resolution
      ndvi = image.expression(
      "(NIR-RED)/(NIR+RED)",
      {
          'RED': image.select('B4').multiply(0.0001),
          'NIR' : image.select('B5').multiply(0.0001)
      
      });# okay
      ndf = ndvi.rename('FNDVI')
      results = ndf.copyProperties(image, ['system:time_start'])
      return image.addBands(results)
  return (images_collection.filterDate(start, endDate).
          filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)).
          map(maskclouds).
          map(fndvi).
          select(['B.*', 'FNDVI']).
          reduce(reducer))


In [333]:
start1 = start
end1 = start.advance(1, 'month')
month1 = monthly_composite()


9

In [334]:
month1.getInfo()

{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:32644',
     'crs_transform': [60, 0, 399960, 0, -60, 3200040],
     'data_type': {'max': 6.553500175476074,
      'min': 0,
      'precision': 'float',
      'type': 'PixelType'},
     'dimensions': [1830, 1830],
     'id': 'B1'},
    {'crs': 'EPSG:32644',
     'crs_transform': [10, 0, 399960, 0, -10, 3200040],
     'data_type': {'max': 6.553500175476074,
      'min': 0,
      'precision': 'float',
      'type': 'PixelType'},
     'dimensions': [10980, 10980],
     'id': 'B2'},
    {'crs': 'EPSG:32644',
     'crs_transform': [10, 0, 399960, 0, -10, 3200040],
     'data_type': {'max': 6.553500175476074,
      'min': 0,
      'precision': 'float',
      'type': 'PixelType'},
     'dimensions': [10980, 10980],
     'id': 'B3'},
    {'crs': 'EPSG:32644',
     'crs_transform': [10, 0, 399960, 0, -10, 3200040],
     'data_type': {'max': 6.553500175476074,
      'min': 0,
      'precision': 'float',
      'type': 'PixelType'},
     'dimens

In [244]:
composite = ee.ImageCollection(images_collection).reduce(ee.Reducer.mean())
#composite.

In [245]:
composite.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 6.553500175476074,
    'min': 0,
    'precision': 'float',
    'type': 'PixelType'},
   'id': 'B8_mean'}],
 'type': 'Image'}

In [247]:
images_collection.getInfo()['features']

[{'bands': [{'crs': 'EPSG:32644',
    'crs_transform': [10, 0, 499980, 0, -10, 3200040],
    'data_type': {'max': 6.553500175476074,
     'min': 0,
     'precision': 'float',
     'type': 'PixelType'},
    'dimensions': [10980, 10980],
    'id': 'B8'}],
  'properties': {'system:index': '20190919T051649_20190919T052955_T44RNS'},
  'type': 'Image'},
 {'bands': [{'crs': 'EPSG:32645',
    'crs_transform': [10, 0, 300000, 0, -10, 3300000],
    'data_type': {'max': 6.553500175476074,
     'min': 0,
     'precision': 'float',
     'type': 'PixelType'},
    'dimensions': [10980, 10980],
    'id': 'B8'}],
  'properties': {'system:index': '20191005T044701_20191005T045002_T45RUN'},
  'type': 'Image'},
 {'bands': [{'crs': 'EPSG:32645',
    'crs_transform': [10, 0, 399960, 0, -10, 3300000],
    'data_type': {'max': 6.553500175476074,
     'min': 0,
     'precision': 'float',
     'type': 'PixelType'},
    'dimensions': [10980, 10980],
    'id': 'B8'}],
  'properties': {'system:index': '20191005T044

## Download Data

In [ ]:
count = int(images_collection.size().getInfo())
print(f"Total number of images: {count}\n")
download_dir = '/content/sentinel_download'
for i in range(0, count): #TODO change number to count
    image = ee.Image(images_collection.toList(count).get(i))
    name = image.get("system:index").getInfo() + ".tif"
    #filename = f'drive/MyDrive/farmhand/sentinel_images/{name}' 
    filename = f"{download_dir}/{name}"
    print(f"Exporting {i + 1}/{count}: {name}")
    # scale = image.projection().nominalScale().multiply(10)
    # download_url = image.getDownloadURL({'region':aoi, 'scale':scale, "name": name, "filePerBand": True})
    # print(download_url)
    geemap.ee_export_image(
        image.select(['B2', 'B3', 'B4', 'B8']).clip(aoi),
        filename=filename,
        region=aoi,
        file_per_band=True
    )
    geemap.ee_export_image(
        image.select(['FNDVI']).clip(aoi),
        filename=filename,
        region=aoi,
        file_per_band=True
    )


Total number of images: 285

Exporting 1/285: 20191103T052001_20191103T052037_T43PGS.tif
Generating URL ...
Please wait ...
Data downloaded to /content/sentinel_download
Generating URL ...
Please wait ...
Data downloaded to /content/sentinel_download
Exporting 2/285: 20191103T052001_20191103T052037_T43PGT.tif
Generating URL ...
Please wait ...
Data downloaded to /content/sentinel_download
Generating URL ...
Please wait ...
Data downloaded to /content/sentinel_download
Exporting 3/285: 20191108T052019_20191108T052749_T43PFT.tif
Generating URL ...
Please wait ...
Data downloaded to /content/sentinel_download
Generating URL ...
Please wait ...
Data downloaded to /content/sentinel_download
Exporting 4/285: 20191113T052051_20191113T053323_T43PFS.tif
Generating URL ...
Please wait ...
Data downloaded to /content/sentinel_download
Generating URL ...
Please wait ...
Data downloaded to /content/sentinel_download
Exporting 5/285: 20191113T052051_20191113T053323_T43PFT.tif
Generating URL ...
Plea

# Visualizing Satellite Imagery

[Indices & composities](https://github.com/sentinel-hub/custom-scripts/tree/master/sentinel-2/false_color_infrared)

[Gee guide for visualization parameters](https://tutorials.geemap.org/Image/image_visualization/)

In [214]:

Map = geemap.Map(location=(15.3510, 76.7318), zoom=10)
vis_params = {
    'bands': ['B8_mean', 'B4_mean', 'B3_mean'],
    'min': 0.2,
    'max': 0.9
}

image = ee.Image(images_collection.toList(100).get(10)) # map one image from collection
Map.addLayer(composite, vis_params, 'false color IR for vegetation') 
#Map.addLayer(farms_bbox, {'color': 'cyan'}, 'farms BBox')
Map.addLayer(aoi, {'color': 'green'}, 'AOI', opacity=0.5)
Map


Map(center=[15.351, 76.7318], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…